In [1]:
import stanza
import os
HOME = os.getcwd()
print(HOME)

from stanza.pipeline.core import DownloadMethod
NLP = stanza.Pipeline('en', download_method=DownloadMethod.REUSE_RESOURCES, use_gpu=True) # to avoid downloading the models every time
import torch

torch.cuda.is_available()


c:\Users\bouab\DEV\see-and-tell\cntell_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\bouab\DEV\see-and-tell


2023-06-27 00:40:52 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

2023-06-27 00:40:53 INFO: Using device: cuda
2023-06-27 00:40:53 INFO: Loading: tokenize
2023-06-27 00:40:57 INFO: Loading: pos
2023-06-27 00:40:58 INFO: Loading: lemma
2023-06-27 00:40:58 INFO: Loading: constituency
2023-06-27 00:40:59 INFO: Loading: depparse
2023-06-27 00:40:59 INFO: Loading: sentiment
2023-06-27 00:41:00 INFO: Loading: ner
2023-06-27 00:41:01 INFO: Done loading processors!


True

# Face recognition

In [2]:
# from src.experimental.exp import get_caption
# from src.face.face_recognition import recognize_faces
# def get_result_image(image_path):
#     # extract the caption
#     caption = get_caption(image_path)
#     # get the faces present in the image 
#     o1 = recognize_faces(image_path, embeddings=bbt_embeddings, display=False)    
#     print(caption)
#     print(o1)
#     return caption, o1


# Test

In [3]:
from pathlib import Path
HOME = os.getcwd()
import json
with open(os.path.join(HOME,'src','TBBT_embeddings_16_160.json')) as f:
  bbt_embeddings = json.load(f)

import os
f = os.path.join(HOME, 'src', 'frames copy')
frames = os.listdir(f)
frames = sorted([os.path.join(f, x) for x in frames], key=lambda x: int(os.path.basename(x)[:-4]))

In [4]:
import pickle 

# with open('all_predictions_1.pickle', 'wb') as handle:
#     pickle.dump(face_pred, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('all_captions_1.pickle', 'rb') as handle:
    captions = pickle.load(handle)


with open('all_predictions_1.pickle', 'rb') as handle:
    face_preds = pickle.load(handle)

In [5]:
# from src.face.face_recognition import recognize_faces
# face_pred = [recognize_faces(os.path.join(f), embeddings=bbt_embeddings) for f in frames[1:]]
assert len(face_preds) == len(captions)

In [6]:
face_preds = [[p[0] for p in ps] for ps in face_preds]

In [7]:
import src.captions_2.captions_improved as ci
gcs, indices = ci.generate_captions(captions, face_preds)

2023-06-27 00:47:54 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

2023-06-27 00:47:54 INFO: Using device: cuda
2023-06-27 00:47:54 INFO: Loading: tokenize
2023-06-27 00:47:55 INFO: Loading: pos
2023-06-27 00:47:56 INFO: Loading: lemma
2023-06-27 00:47:56 INFO: Loading: constituency
2023-06-27 00:47:57 INFO: Loading: depparse
2023-06-27 00:47:58 INFO: Loading: sentiment
2023-06-27 00:47:59 INFO: Loading: ner
2023-06-27 00:48:00 INFO: Done loading processors!


InvalidIndexError: You can only assign a scalar value not a <class 'float'>